In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

importing NGSolve-6.2.2203


setup geometry

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

BaseWebGuiScene

setup FEM space

In [3]:
# viscosity
nu = 0.001
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form

In [4]:
u,p = X.TrialFunction()
v,q = X.TestFunction()

a_stokes = BilinearForm(X)
a_stokes += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a_stokes.Assemble()

setup boundary conditions

In [5]:
gfu = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf,0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene = Draw(velocity, mesh, "vel")
print("->")

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

->


solve equation

In [6]:
res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(X.FreeDofs())
gfu.vec.data += inv * res
scene.Redraw()

In [7]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv * res
    scene.Redraw()
    return res

Drag/ "dissipated energy"

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$

In [8]:
vel = gfu.components[0].vec.data #gfu.vec.data
drag = 0.5*InnerProduct(vel,vel)
print(drag)
# we kinda just need the derivative of u, not this
t = 0.0005310368441135379
print(t-drag)

0.00040430826264929774
0.00012672858146424014


maybe try this to get calculated u vector: f += CoefficientFunction([0, 0, 1])*v*dx

### working drag calculation
taken from navier stokes, is more complicated than needed

In [9]:
drag_test = GridFunction(X)
drag_test.components[0].Set(CoefficientFunction((-1.0,0)), definedon=mesh.Boundaries("cyl"))

res = solveStokes()
drag_val = InnerProduct(res, drag_test.vec)
print(drag_val)

-0.02199565966838881


### get surface area of mesh
(without ball), should stay constant

In [10]:
surf_test = GridFunction(V)
surf_test.Set(CoefficientFunction((1,1))) # , definedon=mesh.Boundaries("cyl")
#print(surf_test.vec/mesh.nedge) #*mesh.nface

In [11]:
#for v in mesh.vertices:
#    print (v, v.point)
nr = []
for e in mesh.Boundaries("cyl").Elements():
    for i in e.vertices:
        nr.append(i.nr)
nr = list(set(nr))    
print(nr)

[4, 5, 6, 7, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]


### Shape derivative $\mathrm{d}J(\Omega)(\mathbf{X}) = \int_{\Omega} \mathbf{S}_1:D \mathbf{X} dx$
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$ <br>
$\mathrm{u}$ and $p$ are the solutions to the stokesflow problem

In [ ]:
# Test and trial functions for shape derrivative, where PSI corresponds to X and 
PHI, PSI = VEC.TnT()

In [ ]:
r = X.TrialFunction()
s = X.TestFunction()
gf = GridFunction(X)
S = BilinearForm(X)
S += (0.5*InnerProduct(grad(u),grad(u))-p*div(v))*Id(2)
S += grad(u).trans*p-grad(u).trans*grad(u)


In [ ]:
# gf, gfp: solution to state&adjoint eq

In [ ]:
def Cost(u):
    return 0.5*InnerProduct(grad(u),grad(u))*dx

def Equation(u,w):
    return (InnerProduct(grad(u),grad(v))+div(u)*q-div(v)*p)*dx

G_pde = Cost(gfu) * Equation(gfu, gfp)

W = V.TestFunction() # X. or V.
dJOmega = LinearForm(VEC)
dfOmega += G_pdre.DiffShape(W)

In [ ]:
"""
grad_S = CoefficientFunction((S.Diff(x),S.Diff(y)))
VEC = H1(mesh, order=1, dim=2)
# grid function for deformation field
gfset = GridFunction(VEC)
gfX = GridFunction(VEC)
scene2 = Draw(gfset)

# Test and trial functions
PHI, PSI = VEC.TnT()

# shape derivative
dJOmega = LinearForm(VEC)
dJOmega += (div(PSI)*f + InnerProduct(grad_f, PSI) )*dx
"""